# Bivariate Gaussian - interactive

In this notebook, we'll get a feel for the two-dimensional Gaussian by varying the covariance matrix, drawing random samples from the resulting distribution, and plotting contour lines of the density.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

### Interactive graphs

http://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider

The function **bivariate_plot** takes as input three parameters that uniquely specify a 2x2 covariance matrix:
* `var1`, the variance of the first feature, `x1`
* `var2`, the variance of the second feature, `x2`
* `corr`, the correlation between `x1` and `x2`

It then depicts a 2-d Gaussian whose mean is the origin and whose covariance matrix is given by these parameters. The display consists of 100 points randomly sampled from the Gaussian, as well as three representative contour lines of the density.

The first line below, **interact_manual**, sets up an interactive widget that allows you to specify the parameters to **bivariate_plot** using sliders, and provides a button to execute the function.

## Set matplotlib default values (2 ways)

 1. rcParams[]
 2. mpl.rc()

### Find default setting values

https://matplotlib.org/api/matplotlib_configuration_api.html#matplotlib.rc

In [ ]:
df_rc = pd.DataFrame([(k, v) for k, v in rcParams.items()], columns=['key', 'value']).set_index('key')

In [ ]:
find = input('Get rcParam containing:\n')
df_rc.filter(regex='.*{}.*'.format(find), axis=0)

### rcParams[]

In [ ]:
from matplotlib.pyplot import rcParams

rcParams['figure.figsize'] = 10, 10
rcParams['left.set_position'] = 'center'

### mpl.rc()

In [ ]:
mpl.rc('figure', figsize=[10., 10.])

# Remove spines
mpl.rc('axes.spines', left=True, top=False, right=False, bottom=False)

# Move left y-axis and bottim x-axis to centre, passing through (0,0)
mpl.rc('axes.spines.set_position', left='center')
# ax.spines['left'].set_position('center')
# ax.spines['bottom'].set_position('center')

# Remove ticks
mpl.rc('xtick.major', top=False, bottom=False)
mpl.rc('ytick.major', left=False, right=False)

In [ ]:
font = {'family' : 'monospace',
        'weight' : 'normal',
        'size'   : 14}

mpl.rc('font', **font)  # pass in the font dict as kwargs

In [ ]:
def get_sigma(var1, var2, corr):
    covariance = corr * np.sqrt(var1) * np.sqrt(var2)
    return [[var1, covariance], [covariance, var2]]

In [ ]:
def print_sigma(sigma):
    np.set_printoptions(precision=2)
    print("Covariance matrix:")
    print(np.around(sigma, decimals=2))

In [ ]:
def adjustFigAspect(fig, aspect=1):
    '''
    Adjust the subplot parameters so that the figure has the correct
    aspect ratio.
    '''
    xsize, ysize = fig.get_size_inches()
    minsize = min(xsize, ysize)
    xlim = .4 * minsize / xsize
    ylim = .4 * minsize / ysize
    if aspect < 1:
        xlim *= aspect
    else:
        ylim /= aspect
    fig.subplots_adjust(left=.5 - xlim,
                        right=.5 + xlim,
                        bottom=.5 - ylim,
                        top=.5 + ylim)

# fig = plt.figure()
# adjustFigAspect(fig, aspect=.5)

In [ ]:
def contour_grid(mu, sigma, lim=10, res=200):
    """To display contour lines, first define a fine grid.
    """
    from scipy.stats import multivariate_normal
    
    # Create grid
    xg = np.linspace(-lim, lim, res)
    yg = np.linspace(-lim, lim, res)
    z = np.zeros((res, res))
    
    # Compute the density at each grid point
    rv = multivariate_normal(mean=mu, cov=sigma)
    for i in range(res):
        for j in range(res):
            z[j, i] = rv.logpdf([xg[i], yg[j]]) 
            
    return xg, yg, z

In [ ]:
@interact(var_x1 = (0.5, 9.), var_x2 = (0.5, 9.), corr=(-0.95, 0.95, 0.05))
def bivariate_plot(var_x1, var_x2, corr):

    # Set parameters of Gaussian
    mu = [0,0]
    
    # Get and print sigma (covariance matrix)
    sigma = get_sigma(var_x1, var_x2, corr)
    # print_sigma(sigma)


    
    # Draw samples from the distribution
    n = 100
    x = np.random.multivariate_normal(mu, sigma, size=n)

    
    # Plot the sampled points as blue dots
    plt.plot(x[: ,0], x[: ,1], 'bo')
    fig, ax = plt.gcf(), plt.gca()
    
    # Set up a plot for the samples and the density contours
    adjustFigAspect(plt.gcf(), aspect=1)
#     lim = 10.0
#     plt.xlim(-lim, lim) # limit along x1-axis
#     plt.ylim(-lim, lim) # limit along x2-axis    
#     plt.axes().set_aspect('equal', 'datalim')
    
    #
    plt.xlabel('var_x1')
    plt.ylabel('var_x2')
    

    # To display contour lines, first define a fine grid
    xg, yg, z = contour_grid(mu, sigma)
           
    # Compute the sign and (natural) logarithm of the determinant of an array
    sign, logdet = np.linalg.slogdet(sigma)
    normalizer = -0.5 * (2 * np.log(6.28) + sign * logdet)
    
    # Now plot a few contour lines of the density
    for offset in range(1, 5):
        plt.contour(xg, yg, z, levels=[normalizer - offset], colors='r', linewidths=2.0, linestyles='solid')

    # Finally, display
    plt.show()

## <font color="magenta">Quick exercise:</font>
Experiment with the widget above to get a sense for how the different parameters influence the shape of the Gaussian. In particular, figure out the answers to the following questions.
* Under what conditions does the Gaussian have contour lines that are perfect circles?
* Under what conditions is the Gaussian tilted upwards?
* Under what conditions is the Gaussian titled downwards?
* Suppose the Gaussian has no tilt, and the contour lines are stretched vertically, so that the vertical stretch is twice the horizontal stretch. What can we conclude about the covariance matrix?

*Note down the answers to these questions: you will enter them later, as part of this week's assignment.*